In [1]:
import itertools
import numpy as np
import pandas as pd
import sys
import os
from datetime import timedelta

print(os.getcwd())

/Users/alexander.prams/hodschallenge


In [2]:
in_file = "challenge/data/device_activations_small.csv"

In [3]:
previous_readings = pd.read_csv(in_file)
print(type(previous_readings['time'][0]))
previous_readings['time'] = pd.to_datetime(previous_readings['time'])
device_list = sorted(previous_readings['device'].unique())
n_devices = len(device_list)
print(type(previous_readings['time'][0]))
print(previous_readings.loc[0, :]['time'])
print(previous_readings.loc[1, :]['time'])
print(previous_readings.loc[2, :]['time'])

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-07-01 04:23:32
2016-07-01 06:52:57
2016-07-01 06:53:00


In [4]:
print("Number of data points: ", len(previous_readings))

Number of data points:  589


In [5]:
def date_time_same_hour(dt1, dt2):
    same_until_hour = dt1.year == dt2.year and dt1.month == dt2.month and dt1.day == dt2.day and dt1.hour == dt2.hour
    if same_until_hour:
        return True
    else:
        return False


In [6]:
# IDEA: ADD HOUR & DAY AS FEATURE?
# 541 hours with positive label
# Weekday as 7 binary inputs?
def preallocate_features(previous_readings):
    n_devices = previous_readings['device'].nunique()
    print('n_devices = ', n_devices)
    tmp_time_stamp = first_time_stamp = pd.to_datetime(previous_readings['time'][0])
    
    unique_hour_date_times = [first_time_stamp.replace(minute=0, second=0)]
    
    hour_interval_start_end = pd.date_range(first_time_stamp.replace(minute=0, second=0), 
                                            previous_readings['time'][len(previous_readings) - 1].replace(minute=0, second=0), 
                                           freq='H')
    
    features = pd.DataFrame(0, index=np.arange(len(hour_interval_start_end)), 
                            columns=['time', 'weekday', 'hour'] + device_list)
    features['time'] = hour_interval_start_end
    return features
#print("Unique hours: ", len(preallocate_features(previous_readings)))
print(preallocate_features(previous_readings))
    

n_devices =  6
                   time  weekday  hour  device_2  device_3  device_4  \
0   2016-07-01 04:00:00        0     0         0         0         0   
1   2016-07-01 05:00:00        0     0         0         0         0   
2   2016-07-01 06:00:00        0     0         0         0         0   
3   2016-07-01 07:00:00        0     0         0         0         0   
4   2016-07-01 08:00:00        0     0         0         0         0   
5   2016-07-01 09:00:00        0     0         0         0         0   
6   2016-07-01 10:00:00        0     0         0         0         0   
7   2016-07-01 11:00:00        0     0         0         0         0   
8   2016-07-01 12:00:00        0     0         0         0         0   
9   2016-07-01 13:00:00        0     0         0         0         0   
10  2016-07-01 14:00:00        0     0         0         0         0   
11  2016-07-01 15:00:00        0     0         0         0         0   
12  2016-07-01 16:00:00        0     0         0 

In [7]:
def preprocess_features_and_labels(previous_readings):
    """
    Generate features so that we have an array with dimensions [T, n_devices, 1]
    T: time
    n_devices: number of devices
    Last dimension: number of activations per hour
    """
    
    # We preallocate to avoid many appends (append copies according to pandas docs, might become an issue/slow for large data)
    features = preallocate_features(previous_readings)
    labels = pd.DataFrame(0, index=np.arange(len(features) - 1), 
                            columns=device_list)
    for index, row in previous_readings.iterrows():
        if index == 0:
            continue
            
        dt = row['time'].replace(minute=0, second=0)
        feature_idx = features.index[features['time'] == dt]
        # Increment device's counter at time
        features.loc[feature_idx, row['device']] += 1
        labels.loc[feature_idx - 1, row['device']] = 1
            
        features['time']
    
    #Second loop, can we improve here?
    for index, row in features.iterrows():
        features.loc[index, 'weekday'] = row['time'].weekday()
        features.loc[index, 'hour'] = row['time'].hour
        
        
    features.drop('time', axis=1, inplace=True)
    
    return (features, labels)

features, labels = preprocess_features_and_labels(previous_readings)

n_devices =  6


In [8]:
print("Features")
print(features)

Features
     weekday  hour  device_2  device_3  device_4  device_5  device_6  device_7
0          4     4         0         0         0         0         0         0
1          4     5         0         0         0         0         0         0
2          4     6         3         0         0         0         0         0
3          4     7        12         0         4         0         6         0
4          4     8         6         2         3         3         5         0
5          4     9         4         1         0         0         0         0
6          4    10         4         5         0         0         0         0
7          4    11         5         1         0         0         0         0
8          4    12         8         4         0         0         0         0
9          4    13         7         5         0         0         0         0
10         4    14         4         3         0         0         0         0
11         4    15         9         4     

In [9]:
print("Labels")
print(labels)

Labels
     device_2  device_3  device_4  device_5  device_6  device_7
0           0         0         0         0         0         0
1           1         0         0         0         0         0
2           1         0         1         0         1         0
3           1         1         1         1         1         0
4           1         1         0         0         0         0
5           1         1         0         0         0         0
6           1         1         0         0         0         0
7           1         1         0         0         0         0
8           1         1         0         0         0         0
9           1         1         0         0         0         0
10          1         1         0         0         0         0
11          1         0         0         0         0         0
12          0         0         0         0         0         0
13          0         0         0         0         0         0
14          0         0         0

In [57]:
def create_timeseries_batches(features, labels, length=31):
    np_features = features.to_numpy()[:-1]
    np_labels = labels.to_numpy()
    
    print(np_features.shape)
    print(np_labels.shape)
    
    np_features = np_features[:length*(len(np_features)//length)]
    np_labels = np_labels[:length*(len(np_labels)//length)]
    
    print(np_features.shape)
    print(np_labels.shape)
    
    feature_batch = np.reshape(np_features, [-1, length, np_features.shape[1]])
    label_batch = np.reshape(np_labels, [-1, length, np_labels.shape[1]])
    return feature_batch, label_batch

feature_batch, label_batch = create_timeseries_batches(features, labels)

(125, 8)
(125, 6)
(124, 8)
(124, 6)


In [91]:
def calc_ratio_positive_outputs_per_device(labels):
    np_labels = labels.to_numpy()[:-1]
    print(np_labels.shape)
    ratio_per_device = np.sum(np_labels, axis=0) / np_labels.shape[0]
    print("Ratio per device: ", ratio_per_device)
    return np.array(ratio_per_device)
ratio_positive_outputs_per_device = calc_ratio_positive_outputs_per_device(labels)

(124, 6)
Ratio per device:  [0.23387097 0.24193548 0.16935484 0.13709677 0.2016129  0.05645161]


In [92]:
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras import objectives
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [108]:
USE_WEIGHTED_LOSS = False

In [109]:
# Create our own weighted loss to combat label imbalance
def weighted_loss(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-10, 1.0-1e-10)
    out = -(y_true * K.log(y_pred) / ratio_positive_outputs_per_device + (1.0 - y_true) * K.log(1.0 - y_pred))
    return K.mean(out, axis=-1)

In [110]:
model = Sequential()
model.add(LSTM(32, input_shape=(None, n_devices + 2), return_sequences=True))
model.add(Dense(n_devices, activation='sigmoid'))
model.compile(loss=weighted_loss if USE_WEIGHTED_LOSS else 'binary_crossentropy', optimizer='adam')
model.fit(feature_batch, label_batch, epochs=300, batch_size=8, verbose=2)

Epoch 1/300
 - 2s - loss: 0.6896
Epoch 2/300
 - 0s - loss: 0.6802
Epoch 3/300
 - 0s - loss: 0.6693
Epoch 4/300
 - 0s - loss: 0.6568
Epoch 5/300
 - 0s - loss: 0.6447
Epoch 6/300
 - 0s - loss: 0.6334
Epoch 7/300
 - 0s - loss: 0.6223
Epoch 8/300
 - 0s - loss: 0.6114
Epoch 9/300
 - 0s - loss: 0.6006
Epoch 10/300
 - 0s - loss: 0.5901
Epoch 11/300
 - 0s - loss: 0.5797
Epoch 12/300
 - 0s - loss: 0.5695
Epoch 13/300
 - 0s - loss: 0.5596
Epoch 14/300
 - 0s - loss: 0.5501
Epoch 15/300
 - 0s - loss: 0.5407
Epoch 16/300
 - 0s - loss: 0.5316
Epoch 17/300
 - 0s - loss: 0.5226
Epoch 18/300
 - 0s - loss: 0.5139
Epoch 19/300
 - 0s - loss: 0.5054
Epoch 20/300
 - 0s - loss: 0.4972
Epoch 21/300
 - 0s - loss: 0.4893
Epoch 22/300
 - 0s - loss: 0.4817
Epoch 23/300
 - 0s - loss: 0.4744
Epoch 24/300
 - 0s - loss: 0.4675
Epoch 25/300
 - 0s - loss: 0.4607
Epoch 26/300
 - 0s - loss: 0.4542
Epoch 27/300
 - 0s - loss: 0.4478
Epoch 28/300
 - 0s - loss: 0.4415
Epoch 29/300
 - 0s - loss: 0.4353
Epoch 30/300
 - 0s - lo

 - 0s - loss: 0.0875
Epoch 239/300
 - 0s - loss: 0.0872
Epoch 240/300
 - 0s - loss: 0.0868
Epoch 241/300
 - 0s - loss: 0.0865
Epoch 242/300
 - 0s - loss: 0.0862
Epoch 243/300
 - 0s - loss: 0.0859
Epoch 244/300
 - 0s - loss: 0.0856
Epoch 245/300
 - 0s - loss: 0.0852
Epoch 246/300
 - 0s - loss: 0.0849
Epoch 247/300
 - 0s - loss: 0.0846
Epoch 248/300
 - 0s - loss: 0.0843
Epoch 249/300
 - 0s - loss: 0.0840
Epoch 250/300
 - 0s - loss: 0.0837
Epoch 251/300
 - 0s - loss: 0.0834
Epoch 252/300
 - 0s - loss: 0.0831
Epoch 253/300
 - 0s - loss: 0.0828
Epoch 254/300
 - 0s - loss: 0.0825
Epoch 255/300
 - 0s - loss: 0.0822
Epoch 256/300
 - 0s - loss: 0.0819
Epoch 257/300
 - 0s - loss: 0.0816
Epoch 258/300
 - 0s - loss: 0.0813
Epoch 259/300
 - 0s - loss: 0.0810
Epoch 260/300
 - 0s - loss: 0.0807
Epoch 261/300
 - 0s - loss: 0.0804
Epoch 262/300
 - 0s - loss: 0.0802
Epoch 263/300
 - 0s - loss: 0.0799
Epoch 264/300
 - 0s - loss: 0.0796
Epoch 265/300
 - 0s - loss: 0.0793
Epoch 266/300
 - 0s - loss: 0.0790

In [111]:
predictions = model.predict(np.expand_dims(features.to_numpy()[:-1], axis=0))
print(np.round(predictions, 1))

[[[0.3 0.2 0.2 0.2 0.3 0.1]
  [0.3 0.1 0.1 0.  0.2 0. ]
  [0.6 0.1 0.1 0.  0.2 0. ]
  [1.  0.9 0.6 0.8 0.7 0.3]
  [1.  0.9 0.2 0.3 0.2 0. ]
  [1.  0.9 0.1 0.1 0.1 0. ]
  [1.  0.9 0.  0.1 0.  0. ]
  [1.  0.9 0.  0.  0.  0. ]
  [1.  0.9 0.  0.  0.  0. ]
  [0.9 0.9 0.  0.  0.  0. ]
  [0.8 0.7 0.  0.  0.  0. ]
  [0.7 0.5 0.  0.  0.  0. ]
  [0.2 0.1 0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0

In [112]:
print("Training accuracy: ", np.sum(np.round(predictions) == labels.to_numpy()) / predictions.size) 
print("Training accuracy per device:: ", np.sum(np.round(predictions) == labels.to_numpy()) / predictions.size) 

print("% of 1 prediction outputs", np.sum(np.round(predictions)) / predictions.size) 
print("% of 1 label outputs", np.sum(np.round(labels.to_numpy())) / labels.to_numpy().size)

Training accuracy:  0.8986666666666666
% of 1 prediction outputs 0.09466666666666666
% of 1 label outputs 0.18


In [105]:
print(labels)

     device_2  device_3  device_4  device_5  device_6  device_7
0           0         0         0         0         0         0
1           1         0         0         0         0         0
2           1         0         1         0         1         0
3           1         1         1         1         1         0
4           1         1         0         0         0         0
5           1         1         0         0         0         0
6           1         1         0         0         0         0
7           1         1         0         0         0         0
8           1         1         0         0         0         0
9           1         1         0         0         0         0
10          1         1         0         0         0         0
11          1         0         0         0         0         0
12          0         0         0         0         0         0
13          0         0         0         0         0         0
14          0         0         0       

In [97]:
model.evaluate(feature_batch, label_batch)

4/4 [==============================] - 1s 138ms/step


0.15141254663467407